# trajectory_compare

compare multiple particle trajectory files and .mat outputs with the same spawn points

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

import numpy as np
import xarray as xr

import utils
from parcels_utils import arrays_to_particleds
from plot_utils import plot_trajectories, plot_particles_nc

In [ ]:
# this should definitely not be a thing
def get_part_hash(ds, i):
    lat = ds["lat"].values[i, 0]
    lon = ds["lon"].values[i, 0]
    return hash((lat, lon))

In [ ]:
# first file in list is used as a reference for every particle's starting point
particle_file_paths = [
    "particle_plume_track.nc",
    "particle_plume_track_totsdlj.nc",
#     "plume_pts_20200810_0030.nc",
]

particle_files = []
for path in particle_file_paths:
    with xr.open_dataset(utils.PARTICLE_NETCDF_DIR / path) as pf:
        particle_files.append(pf)

### gif generation

two simulations in the same plot

In [ ]:
domain = {
    "W": -117.3,
    "E": -117,
    "S": 32.3,
    "N": 32.7,
}

In [ ]:
double_plot_path = utils.create_path(utils.PICUTRE_DIR / "plume_compare_double")
for p in double_plot_path.glob("*.png"):
    p.unlink()

for i in range(particle_files[0].dims["obs"]):
    time_ref = particle_files[0].isel(obs=i)["time"]
    # get the first non-nat time to put as title string
    time = time_ref[np.where(~np.isnat(time_ref))][0].values
    plot_particles_nc(particle_files[0].isel(obs=i), domain, time=time, label="parcels", part_size=1)
    plot_particles_nc(particle_files[1].isel(obs=i), domain, time=time, label="oi", savefile=double_plot_path / f"comp{str(i).zfill(3)}.png", part_size=1)

In [ ]:
utils.create_gif(25, double_plot_path / "*.png", utils.PICUTRE_DIR / f"plume_comp_double.gif")

### singular particle comparisons

In [ ]:
ref_file = particle_files[0]

In [ ]:
particle_ids = np.empty(ref_file.dims["traj"])
for i in range(ref_file.dims["traj"]):
    particle_ids[i] = get_part_hash(ref_file, i)

iterates through the points in the reference netcdf and treats them as spawn points

then it checks all the other netcdf files for points with the same spawn, and then graphs them to compare

saves a separate netcdf file for each spawn and each different particle file given

plots of the comparisons are saved under `snapshots/plume_compare/hash_of_position.png` where hash_of_position is just (lat, lon) tuple hash

In [ ]:
particle_path = utils.create_path(utils.PARTICLE_NETCDF_DIR / "plume_compare")
plot_path = utils.create_path(utils.PICUTRE_DIR / "plume_compare")
for p in plot_path.glob("*.png"):
    p.unlink()
    
for p_id in particle_ids[::25]:
    nc_files = []
    names = []
    num_format = format(p_id, ".0f")
    for pf_i in range(len(particle_files)):
        pf = particle_files[pf_i]
        for i in range(pf.dims["traj"]):
            if p_id != get_part_hash(pf, i):
                continue
            save_path = utils.create_path(particle_path / f"{num_format}")
            time = pf["time"].values[np.newaxis, i]
            lat = pf["lat"].values[np.newaxis, i]
            lon = pf["lon"].values[np.newaxis, i]
            # recreate particle netcdf to drop data we don't need
            ds = arrays_to_particleds(time, lat, lon)
            name = particle_file_paths[pf_i].split("/")[-1].split(".")[0]
            ds.to_netcdf(f"{save_path / Path(name)}.nc")
            nc_files.append(f"{save_path / Path(name)}.nc")
            names.append(name)
            break
    plot_trajectories(nc_files, names, legend=True, savefile=plot_path / f"{num_format}.png")